In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [11]:
# load the trained model, scaler pickle, onehot
model = load_model('model.h5')

# load the encoder and scaler 
with open('onehot_encoder_geo.pkl', 'rb') as f: 
    label_encoder_geo = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f: 
    label_encoder_gender = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [32]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [33]:
# one hot encode Geography
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\DS BY KRISH NAIK\ANN CLASSIFICATION PROJECT\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [35]:
# Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [36]:
# Concatenate the one-hot encoded columns back to the original dataframe and drop the 'Geography' column
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [37]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [38]:
## Scalling the input data
scaled_input = scaler.transform(input_df)

In [39]:
scaled_input

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [41]:
prediction = model.predict(scaled_input)
prediction


1/1 [==============================] - 0s 17ms/step


array([[0.02988861]], dtype=float32)

In [42]:
if prediction[0][0] > 0.5:
    print("The customer is likely to exit the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
